In [1]:
import ee 
import folium
from datetime import datetime as dt
from IPython.display import Image
import pprint

In [2]:
ee.Initialize()

In [3]:
# map log latt :
Ituna_map = folium.Map(location=[41.4, -74], zoom_start=10)
Ituna_map

In [4]:
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")

In [5]:
landsat.getInfo()["properties"]

EEException: Collection query aborted after accumulating over 5000 elements.

In [9]:
landsat.size().getInfo()

1136623

In [10]:
landsat.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [9171, 9161],
   'crs': 'EPSG:32628',
   'crs_transform': [30, 0, 341085, 0, -30, 8807715]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [9171, 9161],
   'crs': 'EPSG:32628',
   'crs_transform': [30, 0, 341085, 0, -30, 8807715]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [9171, 9161],
   'crs': 'EPSG:32628',
   'crs_transform': [30, 0, 341085, 0, -30, 8807715]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [9171, 9161],
   'crs': 'EPSG:32628',
   'crs_transform': [30, 0, 341085, 0, -30, 8807715]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'in

In [14]:
# setting the Area of Interest (AOI)
Ituna_AOI = ee.Geometry.Rectangle([-51.84448, -3.92180,
                                   -52.23999, -4.38201])

In [15]:
# filter area
landsat_AOI = landsat.filterBounds(Ituna_AOI)

In [16]:
# choose dates
# landsat_AOI_filtered = landsat_AOI.filterDate('2019-07-01')
# landsat = landsat.filterDate('2019-07-01','2019-12-01')

In [17]:
landsat_image = landsat_AOI_filtered.first()

NameError: name 'landsat_AOI_filtered' is not defined

In [18]:
landsat_image

NameError: name 'landsat_image' is not defined

In [19]:
pprint.pprint(landsat_image.getInfo())

NameError: name 'landsat_image' is not defined

In [20]:
print(landsat_AOI.size().getInfo())

104


In [21]:
params={'min': 0,
              'max': 1000,
              'dimensions': 512,
              'bands': ['B4', 'B3', 'B2'],
              'region': Ituna_AOI}

In [22]:
landsat_image.getInfo()

NameError: name 'landsat_image' is not defined

In [23]:
landsat_image.bandNames().getInfo()

NameError: name 'landsat_image' is not defined

In [24]:
# the least cloudy image
least_cloudy = ee.Image(landsat_AOI.sort('CLOUD_COVER').first())

In [25]:
least_cloudy.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [7591, 7741],
   'crs': 'EPSG:32622',
   'crs_transform': [30, 0, 321285, 0, -30, -363885]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [7591, 7741],
   'crs': 'EPSG:32622',
   'crs_transform': [30, 0, 321285, 0, -30, -363885]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [7591, 7741],
   'crs': 'EPSG:32622',
   'crs_transform': [30, 0, 321285, 0, -30, -363885]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [7591, 7741],
   'crs': 'EPSG:32622',
   'crs_transform': [30, 0, 321285, 0, -30, -363885]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'in

In [26]:
# how cloudy is it?
print('Cloud Cover (%):', least_cloudy.get('CLOUD_COVER').getInfo())

Cloud Cover (%): 0


In [27]:
# when was this image taken?
date = ee.Date(least_cloudy.get('system:time_start'))
time = date.getInfo()['value']/1000.

In [28]:
dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')

'2017-07-20 13:36:07'

In [29]:
parameters = {'min': 0,
              'max': 1000,
              'dimensions': 512,
              'bands': ['B4', 'B3', 'B2'],
              'region': Ituna_AOI}

In [30]:
Image(url = least_cloudy.getThumbUrl(parameters))

In [18]:
ndvi = least_cloudy.normalizedDifference(['B5', 'B4'])

In [19]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [20]:
palette = ['red', 'yellow', 'green']
ndvi_parameters = {'min': 0,
                   'max': 1,
                   'dimensions': 512,
                   'palette': palette,
                   'region': Ituna_AOI}
Ituna_map.add_ee_layer(ndvi,None ,ndvi_parameters)
Ituna_map